## Agenda
    1. Make sure everyone has the necesary programs:
         - Pyhton
         - Jupyter
         - Github
    2. Intro tp Git
    3. Clone Titanic book
    4. Start cleaning data 

## 5 basic steps to Machine Learning:
#### 1. Data gathering/cleaning/feature development
     2. Model Selection
     3. Fitting the model
     4. Make predicitons
     5. Validate the model

In [ ]:
#just sticking with Pandas today to manipulate and clean the Titanic dataset 
import pandas as pd

In [ ]:
# you will need to map to where your files are located 
train = pd.read_csv(######train.csv')
test = pd.read_csv(#######test.csv')

In [ ]:
train.head()


In [ ]:
test.head()

In [ ]:
# created a dataframe to hold passenger ID and eventually our submission 
Submission = pd.DataFrame()
Submission['PassengerId'] = test['PassengerId']

In [ ]:
# we are going to remove the target (survived) from train and combine train and test so that are feature is consistent
# must be careful when doing this to not drop rows from either set during cleaning
y = train.Survived
combined = pd.concat([train.drop(['Survived'], axis=1),test])

In [ ]:
print(y.shape)
print(combined.shape)

#### We are going to go through each feature one by one and develop our final train and test features

In [ ]:
combined.info()

In [ ]:
combined.head()

In [ ]:
# passenger ID is just an incremented count of passengers, no value so lets drop it

combined = combined.drop('PassengerId', axis= 1)

In [ ]:
combined.head(20)

In [ ]:
# at first glance Name appears to have no value, but notice the title within the name ex. Mrs, Mr, Master, etc
# these may be valuable as it may signify crew or socioeconomic class 
# We need to split out the Title within the Name columnm, we can name it Title
combined['Title'] = combined['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
# and we can drop Name
combined = combined.drop('Name', axis=1)

In [ ]:
combined.head(20)

In [ ]:
# one last action to perform on the Name or now Title, we need to create dummy variables so our value is numeric
combined = pd.get_dummies(combined, columns=['Title'], drop_first=False)

In [ ]:
combined.head()

In [ ]:
# and we must do the same thing to Sex
combined = pd.get_dummies(combined, columns=['Sex'], drop_first=True)

In [ ]:
combined.head()

In [ ]:
# overall age is fine but we are missing values, we need 1309 and we only have 1046. We will do a simple imputation 
# We are also missing 1 fare value, and 2 Embarked values
# We will do a slightly different imputation for each

combined['Age'] = combined.Age.fillna(combined.Age.mean())
combined['Fare'] = combined.Fare.fillna(combined.Age.median())
combined['Embarked'] = combined.Embarked.fillna(combined.Embarked.ffill())

# and we need to create dummy variables for embarked
combined = pd.get_dummies(combined, columns=['Embarked'], drop_first=False)

In [ ]:
combined.head()

In [ ]:
combined.info()

In [ ]:
# we just have ticket and cabin left
# lets first look at the first 20 tickets and see if there may be any value there:

combined.Ticket[:20]

In [ ]:
# there doesn't appear to be anything worth extracting so let's drop Ticket
combined = combined.drop('Ticket', axis=1)

In [ ]:
combined.head()

In [ ]:
# Finally we are left with cabin, lets take a look at the first 50
combined.Cabin[:50]

In [ ]:
# looks like there are a lot of missing values, but they might mean they didn't have a cabin and the significance of A, B, C
# might be something of value.  Let's parse it out and assign the NAN a U for unknown

combined['Cabin'] = combined.Cabin.fillna('U')
combined['Cabin'] = combined['Cabin'].map(lambda c : c[0])

# Finally apply dummy variables to Cabin
combined = pd.get_dummies(combined, columns=['Cabin'], drop_first=False)

In [ ]:
combined.head()

In [ ]:
combined.info()

In [ ]:
# we are good to go.  No missing values and everthing is numeric
# lets put the test and train set back together

# remebering the length of the train set
y.shape

In [ ]:
train = combined.iloc[:891]
test = combined.iloc[891:]
print(train.shape)
print(test.shape)

### We will pick up from here at our next meeting with model selction and introduction to pipelines

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train = train.values
X_test = test.values
y = y.values

In [ ]:
clf = RandomForestClassifier()

In [ ]:
clf.fit(X_train, y)

In [ ]:
Submission['Survived'] = clf.predict(X_test)

In [ ]:
Submission.head()

In [ ]:
# you need to map the submission file to your computer 
Submission.to_csv(##submission.csv', index=False, header=True)